# **Setup**

<font color='black'>In this notebook, you will be using the [`sklearn_crfsuite`](https://sklearn-crfsuite.readthedocs.io/en/latest/) package, which integrates with scikit-learn ([SKL](https://scikit-learn.org/stable/)) to build a predictive model for named entity recognition (NER) tags. This package improves time and memory efficiency, but was developed for SKL version < 0.24. For that reason, SKL is downgraded to 0.24 in this notebook to avoid runtime errors.</font>

In [ ]:
!pip -q install -U scikit-learn==0.23.2 > Log # SKL compatible with sklearn_crfsuite==0.3.6
!pip freeze | grep learn
# !pip show scikit-learn sklearn_crfsuite  # more details about packages

<font color='black'>Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video. The [`warnings`](https://docs.python.org/3/library/warnings.html) package is needed to suppress [`FutureWarning`](https://docs.python.org/3/library/exceptions.html#FutureWarning) alerts rising from the lower version of SKL.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, re, nltk, numpy as np, sklearn_crfsuite as CRF, warnings
from sklearn.model_selection  import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report as rpt  # model's detailed metrics

warnings.simplefilter(action='ignore', category=FutureWarning) # suppress FutureWarning warnings
%time _ = nltk.download(['inaugural'], quiet=True)  # corpora for POS tagging: 'punkt', 'averaged_perceptron_tagger'

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## Loading NER Sample

<font color='black'>In this notebook, you will use the Groningen Meaning Bank ([GMB](https://gmb.let.rug.nl/)) NER annotated dataset (ner_dataset.csv) to train a predictive model to recognize NE tags from sentence/word features, such as capitalization, part of speech (POS), neighboring word attributes, etc. 
    
<font color='black'>In the following cell, load the dataset file, `ner_dataset.csv.gz` into dataframe `df`, where rows represent words and their attributes, such as sentence ID, POS and NE tag. In this notebook, your sample is limited to the first several thousand rows to prevent memory errors.
    
<font color='black'>This model uses the NE tag as an output label. The challenge is that the NE tag is associated with a phrase (i.e., a chunk), but chunking in sentences is an expensive process. The model below will learn to move from parsed words directly to NE tags skipping explicit chunking.

<font color='black'>The `NA` values in `SentID` column are forward-filled and redundant words are removed. </font>

<!-- # sFilePath = 'https://raw.githubusercontent.com/omelnikov/CIS57x/main/ner_dataset.csv.gz' -->

In [ ]:
df = pd.read_csv('ner_dataset.csv.gz', compression='gzip', encoding='ISO-8859-1')[:100000]
df.columns = ['SentID','Word','POS','NE']  # new (shorter) column names
df = df.fillna(method='ffill')   # sequentially propagate (forward-fill) sentence IDs to replace NaNs
df.SentID = df.SentID.apply(lambda s: int(s.replace('Sentence: ', '')))  # remove redundant "Sentence: " terms
df

<font color='black'>You can reproduce any given sentence by concatenating all its words in the `Word` column sequentially.</font>

In [ ]:
print(' '.join(df[df.SentID==1].Word.tolist()))  # we can recover any sentence in its string form

## IOB Notation

<span style="color:black">The GMB dataset uses eight NE tags: gpe (or geopolitical), geo(graphical), org(anization), per(son), tim(e), art(ifact), eve(nt), nat(ural phenomenon). It also uses `IOB` notation for tagging each NE. Each word in a sentence is assigned one tag to indicate what part of an NE tag the word is or whether it's not an NE:
    
1. `I`: Tag is **inside** the chunk, or phrase
2. `O`: Tag is **outside** the chunk
3. `B`: Tag is **beginning** the chunk

<span style="color:black">Run the cell below to visualize `IOB` tagging.</span>

In [ ]:
DssFG = {'I-':'lightgray', 'O-':'gainsboro', 'B-':'black'} # foreground color for tags
DssBG = dict(geo='khaki', gpe='orange', nat='purple', tim='lightblue', \
             per='cornflowerblue', art='lightgreen', eve='goldenrod', org='cyan')

DsHeaderStyle = {'selector': 'th', 'props': [ ('font-size', '5pt'), ('color', 'lightgray')]}
Style = lambda x: f'color: {DssFG.get((x+"-")[:2], "")}; background-color: {DssBG.get(x[2:], "")}'

PrettySent = lambda n=2: df[df.SentID == n ][['Word','NE']].T.\
  style.applymap(Style).set_table_styles([DsHeaderStyle]).\
  set_properties(**{'text-align': 'center'})

for i in [8,15]: PrettySent(i)

<span style='color:black'>Consider the displayed sentences, where each word has either a colored NE tag below it or an uncolored `O`. All contiguous words in an NE share the same color, but only the first tag in the phrase starts with `B`. The rest start with `I`. Thus, in the phrase "*International	Atomic	Energy	Agency*", "*International*" is tagged with `B` and the remaining words tagged with `I`.

<span style="color:black">Next, print the NE tag distribution.
    

In [ ]:
df.NE.value_counts().to_frame().T        # named entity distribution
print(f'Total named entities: {sum(df.NE.str.startswith("B").values)}') # counts NE tags

<span style="color:black">Note that `per` has about the same beginning and inside tags, suggesting that most names contain first and last names in this text. You can make a similar observation about the `org` tag. By contrast, the `geo` tags indicate that the text contains three times more one-word geographical names than multi-word geographical names.
 
<span style="color:black">Use this distribution to build an expectation about the quality of predictions of these tags. More frequent tags (relative to the diversity of your named entities) are likely to show better predictions. If a higher prediction quality is needed for some NE tag, you may add more diverse training examples, look for attributes that describe the related named entities, or focus on tuning your predictive model.

## Building Features
 
<font color="black">While you could force the model to memorize each NE in the training set, this would consume more storage/memory and is unlikely to generalize to new examples. For that reason, you should focus on finding good features of named entities. The model below has a standard set of features, including letter capitalization and POS tags. Features are built for the word in question and its surrounding words.
 
<font color="black">After some experimentation, you might add or remove features to improve a model's performance.  Take care to evaluate poor predictions, then decide on features that might be helpful to improve them. For example, digits are likely to be associated with time, so you may want a feature that indicates whether a word is a digit. Title capitalization may be helpful in identifying proper names, so it may make sense to include a title case feature. If you notice that most organizations follow with a word `'Co.'` or `'Inc.'`, then you might add a feature flagging the presence of these words.

<font color="black">To create features for this text, you can add a field with a tuple of strings (word, POS tag, NER tag). That field is then exported as `LLTsWPE`, or list of lists of tuples of strings of words.

In [ ]:
df['TsWPE'] = list(zip(df.Word, df.POS, df.NE)) # WPE = Word, Part (of speech), (named) Entity
df.head(10).T

LLTsWPE = df.groupby('SentID')['TsWPE'].apply(list).tolist()  # list of lists of WPE tuples
print(LLTsWPE[0])

<font color='black'>You may also want your model to be able to consider a word's neighboring words. The following UDF, `Featurize()` takes a word, its POS, and its relative position in a window of three words. Then, the word's location (`b`=before or `a`=after) is prepended to the key names to distinguish the attributes. 

<font color='black'>Then, the UDF extracts the features and packages them in a dictionary. Notice that these features just reflect reasonable guesses as to what attributes might relate to the given set of NE tags. They form a baseline for measuring future improvements when trying other sets of features.</font>

In [ ]:
def Featurize(sWord='AOL', sPOS='NNP', sLoc='') -> dict():
    ''' Build features from the word and its POS. 
        sLoc: location of word yielding features (relative to word of concern): 
        'b'=before, 'a'=after, ''=current  '''
    return {sLoc + 'D' : sWord.isdigit() * 1, # is the word a digit
            sLoc + 'UC': sWord.isupper()*1,   # is it capitalized
            sLoc + 'TC': sWord.istitle()*1,   # is it in title case
            sLoc + 'T' : sPOS,                # the full POS
            sLoc + 'T2': sPOS[:2]}            # the short POS

print(Featurize(sWord='7', sPOS='NNP', sLoc=''))      # function demo. build NNP features from a word containing a number
print(Featurize(sWord='AOL', sPOS='NNP', sLoc=''))    # function demo. build NNP features from a word containing a capitals
print(Featurize(sWord='have', sPOS='VBP', sLoc='b'))  # function demo. build VBP features from a word containing a verb
print(Featurize(sWord='Boris', sPOS='NNP', sLoc='a')) # function demo. build NNP features from a word containing a title-cased name

<font color='black'>The UDF `WordWindow2Features()` takes a triplet of three words (along with their POS & NE tag attributes) and builds a dictionary of features for each word using the `Featurize()` UDF. It also adds a bias feature, i.e. responsible for training a free parameter (not dependent on any feature), which in linear models captures the additive shift in the distributions fitted to data.</font>

In [ ]:
def WordWindow2Features(
    LTsWPE=[('Yahoo','NNP','B-org'), ('fell','VBD','O'), ('today','NN','B-tim')], i=0) -> dict():
    '''Extract features from the central word, word before 
        and word after (unless central word starts/ends a sentence)
    Inputs:
        LTsWPE:   list of tuples of strings [(word, POS_Tag, NE_Tag), ...]. 
                    These tuples represent sequential words in a sentence.
        i:        integer index of the central word. 
        Returns:  dictionary of features for the central word.      '''
    DsFt = Featurize(LTsWPE[i][0], LTsWPE[i][1]) # extract features from the center word and its POS tag
    DsFt.update({'b':1.})  # add a bias parameter to raise model flexibility
    DsFt.update(Featurize( LTsWPE[i-1][0], LTsWPE[i-1][1], sLoc='b') if i>0 else {'BOS':1}) # a word before center word
    DsFt.update(Featurize( LTsWPE[i+1][0], LTsWPE[i+1][1], sLoc='a') if i<(len(LTsWPE)-1) else {'EOS':1}) # a word after center word
    return DsFt  # return dictionary of string keys with feature values (heterogeneous types)

print(WordWindow2Features(i=0))  # Featurize 1st word in "Yahoo fell today"
print(WordWindow2Features(i=1))  # Featurize 2nd word in "Yahoo fell today"
print(WordWindow2Features(i=2))  # Featurize last word in "Yahoo fell today"

<font color='black'>The following two UDFs encode a sentence (a list of words with their POS and NE tags) and prepare a series of labels for each word in a sentence, i.e., a list of tuples of strings.</font>

In [ ]:
Sent2Features = lambda LTsWPE: [WordWindow2Features(LTsWPE, i) for i in range(len(LTsWPE))]
Sent2Labels   = lambda LTsWPE: [NE for Word, POS, NE in LTsWPE]  # pull out labels for each word in a sentence

print(' '.join(list(zip(*LLTsWPE[0]))[0])) 
print(LLTsWPE[0][5:7])                  # 5th and 6th WPE elements to be featurized
print(Sent2Features(LLTsWPE[0][5:7]))   # a list of feature dictionaries for 5th and 6th WPE elements
print(Sent2Labels(LLTsWPE[0][5:7]))     # a list of named entity (NE) labels for 5th and 6th WPE elements

## Splitting Observations into Train and Test Sets

<font color='black'>Finally, create input features from each sentence with their corresponding labels. Then, split these into training and testing sets.

In [ ]:
X = [Sent2Features(s) for s in LLTsWPE]
Y = [Sent2Labels(s) for s in LLTsWPE]
tX, vX, tY, vY = train_test_split(X, Y, test_size=0.25, random_state=0)
print('tY (train NE labels):', tY[0])
print('tX (train features): ', str(tX[0])[:300], '...')

## Training a Model

<font color='black'>Now, you are ready to train a **conditional random fields** ([CRF](https://repository.upenn.edu/cgi/viewcontent.cgi?article=1162&context=cis_papers)) model just like you would train any other model in SKL. You can tune the parameters below to improve the model performance, although using the full given sample and increasing iterations can result in an even greater performance improvement.

<font color='black'>The CRF model estimates the probability $\mathbb P[Y|X]$, where $Y$ and $X$ are unobserved and observed nodes in an undirected graphic, respectively. This model is often used in NER and similar tasks because of its superior (compared to Hidden Markov Model (HMM), for example) ability to segment and label text. Notice that segmenting (or chunking herein) is performed by CRF itself and not apriori.

<font color='black'>The tuning parameters available for the [`CRF()`](https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite) object are listed below. A few are already modified in the following cell, but you are encouraged to explore the others.

* <font color='black'>`max_iterations` is currently set to 10 to avoid time-consuming fitting. If the optimal number of iterations is much larger, fewer iterations will result in poorer model performance.
* <font color='black'>`algorithm='lbfgs'` is a limited memory version of the popular Broyden–Fletcher–Goldfarb–Shanno (BFGS) iterative algorithm for solving unconstrained nonlinear optimization tasks. It is a variant of the gradient descent method.
* <font color='black'>`c1` and `c2` are coefficients for lasso and ridge regularizations which control the magnitude of the learned parameters. Larger regularization coefficients (unequally) forces model parameters towards zero, thereby lowering their effect in the model. These can be used to control model stability (i.e., bias-variance tradeoff of the output performance).
* <font color='black'>`all_possible_transitions=True` forces the model to generate transition features, which were not specified in advance. Thus, given $L$ features, the model will try to fit $L^2$ extended features (original plus transitions).

In [ ]:
# ?CRF.CRF   # show help manual for the hyper-parameters
crf = CRF.CRF(max_iterations=10, algorithm='lbfgs', 
                           c1=0.1, c2=0.1, all_possible_transitions=True, verbose=0)
crf.fit(tX, tY)  # fit the model on training inputs and outputs

## Predict NE Tags

<font color='black'>Now you can use the trained model `crf` to make predictions for each word in a sentence using the test set `vX`. Notice that the predicted output is no different than the actual output.

In [ ]:
pY = crf.predict(vX)                # predicted NE labels
print('Actual NER tags:', vY[0])    # test (or validation) NE labels
print('Predicted  tags:', pY[0])

## Performance Report
 
<font color='black'>You can derive a more complete performance report using the built-in `flat_classification_report()` function. Here, evaluate which labels are performing relatively poorly based on their precision/recall or f1 metrics. 
 
<font color='black'>To improve the performance of the specific NE tags (or all of them), you could:
 
1. Add more diverse examples for the NE categories
1. Evaluate examples in the underperofrming NE category and deduce new features that appear to describe the considered named entities
1. Tune the hyperparameters in the machine learning model

In [ ]:
labels = list(crf.classes_)
labels.remove('O')    # suppress statistics for "O" tags
pd.DataFrame(rpt(vY, pY, labels=labels, output_dict=True, zero_division=0)).round(2)

## NER on New Text

<font color='black'>Consider the `sDoc` below. After some preprocessing and POS tagging, you can use the model above to predict NE tags.

In [ ]:
sDoc = """Three more countries have joined an "international grand 
    committee" of parliaments, adding to calls for Facebook’s boss, Mark 
    Zuckerberg, to give evidence on misinformation to the coalition. Brazil, 
    Latvia and Singapore bring the total to eight different parliaments across 
    the world, with plans to send representatives to London on 27 November 
    with the intention of hearing from Zuckerberg. Since the Cambridge 
    Analytica scandal broke, the Facebook chief has only appeared in front of 
    two legislatures: the American Senate and House of Representatives, and 
    the European parliament. Facebook has consistently rebuffed attempts from 
    others, including the UK and Canadian parliaments, to hear from Zuckerberg. 
    He added that an article in the New York Times on Thursday, in which the 
    paper alleged a pattern of behaviour from Facebook to "delay, deny and 
    deflect" negative news stories, "raises further questions about how recent 
    data breaches were allegedly dealt with within Facebook." """
sDoc = re.sub(r'\n', '', sDoc)
LsWords = nltk.word_tokenize(sDoc)
LTsWP = nltk.pos_tag(LsWords)
print(LTsWP[:10])

<font color='black'>Featurize the text, as before.

In [ ]:
LDsFeatures = [Sent2Features(LTsWP)]
print(LDsFeatures[0][0])

<font color='black'>Now, the trained model can take the features, i.e. the list of dictionaries, and produce NE tags for each word. Since you passed the whole document at once, the returned list contains a single list element, which you can extract with index 0. 

In [ ]:
LsDocLabels = crf.predict(LDsFeatures)[0] # use trained model to predict NER tags
print(LsDocLabels[10:20])

<font color='black'>Next, generate a more complete output with words and their NE tags. 

In [ ]:
pd.set_option('display.max_columns', 400)
LTsWE = [(Word, NE) for Word, NE in zip(LsWords, LsDocLabels)]
pd.DataFrame(LTsWE).T

<font color='black'>Note that there is still room for model improvement. In particular, `international grand committee` is not tagged, while `Facebook` is tagged as `geo`.

<font color='black'>You can only evaluate the tagged entities. `Mark Zuckerberg` is correctly identified as a person, and `Brazil`, `Latvia`, `Singapore`, and `London` are correctly identified as geographical names.

In [ ]:
dfNE = pd.DataFrame(LTsWE, columns=['word', 'NER_tag']).query('NER_tag!="O"').set_index('word')  # simple display fails to combine named entities
dfNE.T

<font color='black'>Next, combine words that belong to a single entity and leave only their beginning NE tag, for convenience.

In [ ]:
dfNE['id'] = list(range(len(dfNE)))                         # assign integer IDs to each row, 0, 1, 2, 3, ..., len(dfNE)
dfNE.loc[dfNE.NER_tag.str.startswith('I-'), 'id'] = np.NaN  # replace id's with NaN wherever NER_tag starts with 'I-'
dfNE.loc[dfNE.NER_tag.str.startswith('I-'), 'NER_tag'] = '' # replace NER_tag's with '' wherever NER_tag starts with 'I-'

# forward-fill column gaps with a previous values of the column
# Eg. column values [a,b,,,g] are changed to [a,b,b,b,g]
dfNE = dfNE.fillna(method='ffill')                          

# concatenate all values that have matching id's. Set resulting phrases as an index named 'word'
dfNE = dfNE.reset_index().groupby('id').transform(lambda x: ' '.join(x)).drop_duplicates().set_index('word')
dfNE.T        # print transposed dataframe

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

<font color='black'>Now you will practice identifying NE tags in a new document using the model you pretrained above.
    
As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## Task 1

Use NLTK's inaugural speeches to retrieve the words of President Biden's 2021 speech. Assign POS tags to these words and save these as an `LTsWP1` list of tuples of strings of words and POS tags (similar to `LTsWP` above).

<b>Hint:</b> You can use <code>nltk.corpus.inaugural.fileids()</code> to list all inaugural speech file IDs. Then use <code>words()</code> method of <code>nltk.corpus.inaugural</code> object to retrieve the list of words of the speech. Use <code>nltk.pos_tag()</code> method to retrieve POS tags.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
[s for s in nltk.corpus.inaugural.fileids() if 'Biden' in s]
LTsWP1 = nltk.pos_tag(nltk.corpus.inaugural.words('2021-Biden.txt'))
print(LTsWP1[:10])
</pre>
</details> 
</font>

<hr>

## Task 2

Use `Sent2Features()` UDF to create `LDsFeatures1`, a dictionary of features from `LTsWP1`.

<b>Hint:</b> This is similar to the application of <code>Sent2Features()</code> above. 

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
LDsFeatures1 = Sent2Features(LTsWP1)
print(LDsFeatures1[0])
</pre>
</details> 
</font>

<hr>

## Task 3

Use the trained `crf` model to predict a label for each word in the inaugural speech based on `LDsFeatures1` input to the trained model.

<b>Hint:</b> This is similar to the application of the <code>crf.predict()</code> above. Note that <code>predict</code> expects a list of lists, so you may need to wrap `LDsFeatures1` into a list.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
LsDocLabels1 = crf.predict([LDsFeatures1])[0]   # use trained model to predict NER tags[0]
print(LsDocLabels1[10:20])

df1 = pd.DataFrame(LTsWP1)
df1['NE'] = LsDocLabels1
df1.T
</pre>
</details> 
</font>

<hr>